In [3]:
import numpy as np
import cvxpy as cp
import pickle

from problems.data_generation_Price_Setting import generate_problem_data
from problems.problem_definition_Price_Setting import BilevelProblem
from problems.problem_definition_Price_Setting_regularized import BilevelProblem_regularized
from algorithms.barrier_blo import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA
from algorithms.BSG import BSG

n = 60
hparams = {
    'barrier_blo': {
        'M': 0.00001,
        't': 0.0001,
        'alpha_xy': 0.05,
        'alpha_T': 1,
        'epsilon_xy': 0.05,
        'epsilon_T': 0.05,
        'inner_max_iters': 1000000,
        'outer_max_iters': 100000
    },
    'blocc': {
        'gamma': 10,
        'alpha_T': 0.05,
        'alpha_g_xy': 10.0,
        'alpha_F_xy': 0.005,
        'beta_g_xy': 0.000005,
        'beta_F_xy': 0.05,
        'epsilon_xt': 0.1,
        'epsilon_inner_xy_g': 0.01,
        'epsilon_outer_xy_g': 0.01,
        'epsilon_inner_xy_F': 0.01,
        'epsilon_outer_xy_F': 0.01,
        'maxmin_g_outer_max_iters': 5000000,
        'maxmin_F_outer_max_iters': 5000000,
        'maxmin_g_inner_max_iters': 5000000,
        'maxmin_F_inner_max_iters': 5000000,
        'main_max_iters': 1000
    },
    'bic_gaffa':{
        'alpha':0.000001,
        'c':1,
        'tau':1.1,
        'gamma_1':10,
        'gamma_2':1,
        'eta':1,
        'r':1,
        'epsilon':0.1,
        'max_iters':300
    },
    'BSG':{
        'alpha_x':0.00005,
        'alpha_y':0.0005,
        'alpha_z':0.0005,
        'epsilon_x':0.1,
        'epsilon_y':0.1,
        'epsilon_z':0.1,
        'max_iters_x':1000000,
        'max_iters_y':1000000,
        'max_iters_z':1000000
    },
    'IGD': {
        'M': 1e-3,
        't': 1e-3,
        'alpha_x': 0.001,
        'alpha_y': 0.001,
        'epsilon_x': 1e-3,
        'epsilon_y': 1e-3,
        'inner_max_iters': 1000,
        'outer_max_iters': 1000
    }
}

seed = 9

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)
problem_regularized = BilevelProblem_regularized(data)

max_elapsed_time = 1200000000
T_init = np.zeros(n)
x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
x_g_init = np.zeros(n)
x_F_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)

In [4]:
n = 60
max_elapsed_time = 1200000000
for seed in range(10):
    data = generate_problem_data(n, seed)
    problem = BilevelProblem(data)
    problem_regularized = BilevelProblem_regularized(data)

    T_init = np.zeros(n)
    x_init = np.zeros(n)
    y_init = np.zeros(n)
    z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    theta_init = np.zeros(n)
    x_g_init = np.zeros(n)
    x_F_init = np.zeros(n)
    y_g_init = np.zeros(n)
    y_F_init = np.zeros(n)
    mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    
    ## barrier algo
    barrier_algo = BarrierBLO(problem, hparams)
    T_opt_barrier, x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(T_init, x_init, y_init, max_elapsed_time)
    f_barrier_num = problem.f(T_opt_barrier, x_opt_barrier, y_opt_barrier)
    print(f_barrier_num)
    
    x_original_opt_barrier, y_original_opt_barrier = barrier_algo.Interior_inner_loop(T_opt_barrier)
    f_barrier_theo = problem.f(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    g_barrier_theo = problem.g(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    g_barrier_num = problem.g(T_opt_barrier, x_opt_barrier, y_opt_barrier)

    h1_values = [problem.h_1(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
    h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

    # print(f"x_opt_barrier={x_original_opt_barrier},y_opt_barrier={y_original_opt_barrier}")

    print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
    print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
    print(f"feasibility_violation={h_total_barrier}")
    
    # ## BLOCC
    # blocc_algo = BLOCC(problem_regularized, hparams)
    # x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(T_init, x_g_init, x_F_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time)
    # f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
    # print(f_blocc_num)
    
    ## saving results
    results = {
        'barrier': {
            'x_opt': x_opt_barrier,
            'y_opt': y_opt_barrier,
            'history': history_barrier,
            'y_original_opt': y_original_opt_barrier,
            'f_theoretical_opt': f_barrier_theo,
            'f_numerical': f_barrier_num,
            'g_theoretical_opt': g_barrier_theo,
            'g_numerical': g_barrier_num,
            'h_total': h_total_barrier
        }
    }

    save_filename = f'results_PS_seed_{seed}.pkl'

    with open(save_filename, 'wb') as f:
        pickle.dump(results, f)

    print(f"Results saved to {save_filename}")

Outer iteration 1
  Gradient norm = 185.208225317122
  Gradient norm = 70.3764752868414
  Gradient norm = 85.12505006732911
  Gradient norm = 86.44460597025531
  Gradient norm = 82.31204555019953
  Gradient norm = 78.37646505999844
  Gradient norm = 74.62844071566401
  Gradient norm = 71.05902616857459
  Gradient norm = 67.65971004411168
  Gradient norm = 64.42239018588792
  Gradient norm = 61.33935229184547
  Gradient norm = 58.403250592850625
  Gradient norm = 55.6070898634405
  Gradient norm = 52.94420847970889
  Gradient norm = 50.408262381214016
  Gradient norm = 47.993209850033494
  Gradient norm = 45.6932970457983
  Gradient norm = 43.503044248483334
  Gradient norm = 41.41723276835505
  Gradient norm = 39.43089248706497
  Gradient norm = 37.53928999728915
  Gradient norm = 35.737917310745175
  Gradient norm = 34.022481106411014
  Gradient norm = 32.38889249255974
  Gradient norm = 30.833257257595612
  Gradient norm = 29.351866586129926
  Gradient norm = 27.94118821789548
  Grad

In [5]:
for seed in range(10):
    filename = f'results_PS_seed_{seed}.pkl'
    with open(filename, 'rb') as f:
        results = pickle.load(f)
    history = results['barrier']['history']
    if history:
        last_time = history[-1]['time']
        print(f"Seed {seed} - Total Time: {last_time:.2f} second")

Seed 0 - Total Time: 10.95 second
Seed 2 - Total Time: 5.69 second
Seed 3 - Total Time: 9.39 second
Seed 4 - Total Time: 10.31 second
Seed 5 - Total Time: 17.40 second
Seed 6 - Total Time: 4.03 second
Seed 7 - Total Time: 6.75 second
Seed 8 - Total Time: 5.18 second
Seed 9 - Total Time: 6.12 second


In [16]:
hparams['blocc'] = {
            'gamma': 10,
            'alpha_T': 0.05,
            'alpha_g_xy': 10.0,
            'alpha_F_xy': 0.005,
            'beta_g_xy': 0.000002,
            'beta_F_xy': 0.05,
            'epsilon_xt': 0.1,
            'epsilon_inner_xy_g': 0.01,
            'epsilon_outer_xy_g': 0.01,
            'epsilon_inner_xy_F': 0.01,
            'epsilon_outer_xy_F': 0.01,
            'maxmin_g_outer_max_iters': 5000000,
            'maxmin_F_outer_max_iters': 5000000,
            'maxmin_g_inner_max_iters': 5000000,
            'maxmin_F_inner_max_iters': 5000000,
            'main_max_iters': 1000
        }

blocc_algo = BLOCC(problem_regularized, hparams)
x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(T_init, x_g_init, x_F_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time)
f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
print(f_blocc_num)

Main iteration 1
Inner loop for L_g converges when iter = 340
Outer iter for L_g=0, Projected Gradient norm w.r.t mu of L_g=149102.8765360622
Inner loop for L_g converges when iter = 351
Outer iter for L_g=1, Projected Gradient norm w.r.t mu of L_g=90328.3095260922
Inner loop for L_g converges when iter = 326
Outer iter for L_g=2, Projected Gradient norm w.r.t mu of L_g=54729.35793883238
Inner loop for L_g converges when iter = 301
Outer iter for L_g=3, Projected Gradient norm w.r.t mu of L_g=33230.096666152094
Inner loop for L_g converges when iter = 276
Outer iter for L_g=4, Projected Gradient norm w.r.t mu of L_g=20392.557498720675
Inner loop for L_g converges when iter = 250
Outer iter for L_g=5, Projected Gradient norm w.r.t mu of L_g=12920.858337826578
Inner loop for L_g converges when iter = 223
Outer iter for L_g=6, Projected Gradient norm w.r.t mu of L_g=8812.56629446971
Inner loop for L_g converges when iter = 196
Outer iter for L_g=7, Projected Gradient norm w.r.t mu of L_g=

KeyboardInterrupt: 